In [2]:
import pandas as pd
#Importamos los datos del dataset 
data = pd.read_csv('Grupo_2_data_CO2_short.csv')

#Eliminamos los espacios en blancos en los nombres de las columnas para facilitar la manipulación de los datos
nombre_colum= list(data.columns.values)
for i in range(len(nombre_colum)):
    nombre_colum[i]=nombre_colum[i].strip() 
data.columns = nombre_colum
#Mostramos los datos
data

,Yr,Mn,Date,CO2
0,1957,3,1957.2027,315.45
1,1957,4,1957.2877,315.74
2,1957,5,1957.3699,315.84
3,1957,6,1957.4548,315.28
4,1957,7,1957.5370,-99.99
...,...,...,...,...
758,2020,5,2020.3716,418.17
759,2020,6,2020.4563,416.11
760,2020,7,2020.5383,407.54
761,2020,8,2020.6230,406.61


Condiciones de los datos: 

Identifique el intervalo de tiempo continuo (meses consecutivos) más extenso que cumple tres condiciones:

- El intervalo termina con la última observación de Septiembre 2020 (último dato del intervalo)
- El intervalo comienza con una observación con un dato válido de CO2 (condición 1 para el primer dato)
- En todo el intervalo más del 95% de las observaciones tienen un dato de CO2 válido (condición 2)

In [3]:
#verificación de existencia de valores nulos
print(data.isnull().sum()) 
print("Cantidad total de valores nulos ", data.isnull().sum().sum()) 

Yr      0
Mn      0
Date    0
CO2     0
dtype: int64
Cantidad total de valores nulos  0


In [4]:
#Atendiendo la condición 1, el último dato del dataset coincide con la última observación del 2020
data.loc[data['Mn'] == 9 ]
print(data.iloc[762])


Yr      2020.0000
Mn         9.0000
Date    2020.7077
CO2      410.8700
Name: 762, dtype: float64


In [12]:
#Creación de la dataset con la cual vamos a trabajar.

pos=0; neg=0; por=0

#Recorremos todos los datos del dataset
for i in range(len(data)):
  #Primera condición, se crea el intervalo siempre y cuando CO2 sea un dato válido
  if data.CO2.values[i] > 0: 
    #Creo el intervalo desde la posición 'i' hasta la última posición del dataset (2da condición)
    intervalo=data[i:]
    #Recorro los datos del intervalo recién creado contando los valores positivos y negativos
    for j in range(len(intervalo)): 
        #Si el dato en la posición 'j' es mayor que 0, sumo 1 a la variable pos
        if intervalo.CO2.values[j] >= 0:
            pos=pos + 1
        #Si el dato es negativo, sumo 1 a la variable neg
        else:
            neg=neg+1
    #Calculo porcentaje de los valores pos
    por= (pos / (pos+neg) )*100
    #Reinicio contadores
    pos=0; neg=0; 
    #Comprobamos la 3ra condición, apenas se cumpla que el porcentaje 'por' es mayor que 94, se guarda el dataset en la posición 'i' y se detiene el bloque
    if por > 94: #borré el 
        data_f=data[i:]
        break 
print('Intervalo final: \n', data_f)
             

Intervalo final: 
        Yr  Mn       Date     CO2
292  1981   7  1981.5370  337.94
293  1981   8  1981.6219  336.44
294  1981   9  1981.7068  -99.99
295  1981  10  1981.7890  339.23
296  1981  11  1981.8740  340.16
..    ...  ..        ...     ...
758  2020   5  2020.3716  418.17
759  2020   6  2020.4563  416.11
760  2020   7  2020.5383  407.54
761  2020   8  2020.6230  406.61
762  2020   9  2020.7077  410.87

[471 rows x 4 columns]


In [6]:
#Añadimos una nueva columna que contendrá los nuevos valores de CO2
data_f['newCO2']= data_f.CO2.values
data_f

C:\Users\Jose\AppData\Local\Temp\ipykernel_20144\1912727356.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_f['newCO2']= data_f.CO2.values


,Yr,Mn,Date,CO2,newCO2
292,1981,7,1981.5370,337.94,337.94
293,1981,8,1981.6219,336.44,336.44
294,1981,9,1981.7068,-99.99,-99.99
295,1981,10,1981.7890,339.23,339.23
296,1981,11,1981.8740,340.16,340.16
...,...,...,...,...,...
758,2020,5,2020.3716,418.17,418.17
759,2020,6,2020.4563,416.11,416.11
760,2020,7,2020.5383,407.54,407.54
761,2020,8,2020.6230,406.61,406.61


In [7]:
#Comprobamos la posición y cantidad de los valores negativos dentro de la columna 'CO2'
pos_neg=[]
for i in range(len(data_f)):
    if data_f.CO2.values[i] < 0:
        pos_neg.append(i)
print(pos_neg)
print('Cantidad de valores negativos', len(pos_neg))

#revisé los datos y me encontré con 200 valores negativos, no 28

[2, 7, 13, 15, 53, 54, 61, 114, 136, 173, 185, 208, 258, 280, 305, 316, 320, 326, 336, 352, 362, 365, 388, 402, 415, 449, 458, 462]
Cantidad de valores negativos 28


In [8]:
### Creación de bloques ###
###########################

#Inicializamos contadores:
i=0; j=0; bloque=[]; date=[]; t=0
#Ejecutamos los pasos mientras i sea menor al tamaño del dataset recién creado
while i < len(data_f):   
    #Si el valor en la posición 'i' es negativo, creamos el bloque
    if data_f.CO2.values[i] < 0:
        #añadimos el valor inmediatamente anterior al valor negativo como inicio del bloque
        bloque.append(data_f.CO2.values[i-1])
        #añadimos la fecha correspondiente a ese valor a un bloque de fechas
        date.append(data_f.Date.values[i-1])
        #Guardamos la posición del primer valor negativo dentro del bloque
        pos=i
        #Creamos un nuevo ciclo que ejecute los procesos siguientes siempre y cuando los valores a recorrer sean negativos, partimos de la posición 'i'
        j=i
        while data_f.CO2.values[j] < 0: 
            #guardamos los valores negativos en el bloque
            bloque.append(data_f.CO2.values[j])
            #guardamos las fechas correspondientes
            date.append(data_f.Date.values[j]) 
            j=j+1
        #Si se deja de cumplir la condición, es porque estamos en un positivo, guardamos ese valor positivo como final del intervalo y guardamos su fecha correspondiente
        else:
            bloque.append(data_f.CO2.values[j])
            date.append(data_f.Date.values[j])
            i=i+1
    ##### Ajuste de los nuevos valores:  #####
    ################################################

    #Ejecutamos el proceso solo si el bloque no está vacío
    if bool(bloque) == True:
        #Límite inferior del bloque
        bloque_inf= bloque[0]
        #Límite superior del bloque
        bloque_sup= bloque[-1]
        #Fecha inferior del bloque de fechas
        date_inf= date[0]
        #Fecha superior del bloque de fechas
        date_sup= date[-1]
         
        #Si el tamaño del bloque es 3, el ajuste de valores es inmediato.
        if len(bloque) == 3:
            t= data_f.Date.values[pos]
            data_f.newCO2.values[pos] =(1/0.3343) * ( (t- date_inf)* bloque_sup - (t - date_sup)* bloque_inf) 
        #Si el tamaño del bloque es 4, el ajuste de valores se hace para dos valores consecutivos: 'pos' y 'pos + 1'
        else:
            t= data_f.Date.values[pos]
            data_f.newCO2.values[pos] =(1/0.3343) * ( (t-date_inf)* bloque_sup - (t - date_sup)* bloque_inf)

            t= data_f.Date.values[pos+1]
            data_f.newCO2.values[pos+1] =(1/0.3343) * ( (t-date_inf)* bloque_sup - (t - date_sup)* bloque_inf)
    #Vaciamos las variables al término del proceso para que no se concatenen al ejecutar el proceso nuevamente.   
    bloque=[]; date=[]
    i=i+1; j=0;

In [9]:
#Nueva tabla actualizada, comprobar con los valores de la lista "pos_neg" creada anteriormente los nuevos valores.
#1, 9, 15, 17, 55, 56, 63, 116, 138, 175, 187, 210, 260, 282, 307, 318, 322, 328, 338, 354, 364, 367, 390, 404, 417, 451, 460, 464
data_f.iloc[17]

Yr        1982.0000
Mn          12.0000
Date      1982.9562
CO2        343.0900
newCO2     343.0900
Name: 309, dtype: float64

In [10]:
#Comentario, creo y solamente creo, que el ajuste de los nuevos valores está mal,
# debería ser algo que ronde entre los 300 (como sale en la guía), pero el proceso como tal anda de lolo manolo.

In [11]:
#Ítem 3